### Partial correlation analysis on cryptocurrencies

 I will use graphical network analysis technique to understand the relationships between different cryptocurrencies while holding all other cryptocurrencies as constants.
 
 The inverse of the correlation matrix, which is called the precision matrix (denoted by Θ), is used to captures pairwise correlations between cryptocurrencies by removing the effects of all other variables. This helps us understand conditional or hidden dependencies between cryptocurrencies, and in practice, can help us to increase diversification & as such our portfolio's Sharpe Ratio within this asset class.

**Note** : You can only run the data downloading cell **once**. If you need to run it again, please restart and run all from the beginning.
 

In [14]:
#Download the required libraries 
import numpy as np
import pandas as pd
import yfinance as data
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection

#Graphical Analysis libraries
from sklearn.covariance import GraphicalLassoCV
from sklearn.covariance import GraphicalLasso
import csv
from Graphical_Analysis_functions import *

I will stick to top 30 cryptocurrencies as of 12 Jan 2024 from Yahoo Finance: https://finance.yahoo.com/u/yahoo-finance/watchlists/crypto-top-market-cap/

Stablecoins and wrapped version of same tokens have been deliberately excluded, so only 22 top cryptocurrencies by market cap is included

In [2]:
#Import any cryptocurrency symbols. You can also replace this with any kind of securities available in Yfinance
symbol = ['BTC-USD', 'ETH-USD', 'BNB-USD', 'SOL-USD', 'XRP-USD', 'ION-USD', 'ADA-USD', 'AVAX-USD', 'DOGE-USD', 'DOT-USD', 'TRX-USD', 'MATIC-USD', 'LINK-USD',
'SHIB-USD', 'ICP-USD', 'BCH-USD', 'LTC-USD', 'ETC-USD', 'ATOM-USD', 'LEO-USD', 'OP-USD', 'WEOS-USD']
start_date = '2018-01-01' 
end_date = '2024-01-12'

In [12]:
#Once you have ran this cell, you will need to restart from the beginning if you want to rerun it
data = data.download(tickers = symbol, start = start_date, end = end_date)
data.columns.names = ['Attribute', 'Symbols']
data.tail()

[*********************100%***********************]  22 of 22 completed


Attribute  Adj Close                                                \
Symbols      ADA-USD   ATOM-USD   AVAX-USD     BCH-USD     BNB-USD   
Date                                                                 
2024-01-06  0.523103   9.830917  34.624180  236.081741  307.794495   
2024-01-07  0.494174   9.408579  33.629654  232.168503  302.890533   
2024-01-08  0.540936   9.956762  36.082607  253.899994  304.500397   
2024-01-09  0.511527   9.657955  34.695011  245.189453  300.976105   
2024-01-10  0.566647  10.380351  38.472389  254.239105  305.096832   

Attribute                                                             ...  \
Symbols          BTC-USD  DOGE-USD   DOT-USD    ETC-USD      ETH-USD  ...   
Date                                                                  ...   
2024-01-06  43989.195312  0.080664  7.138831  19.882305  2241.624756  ...   
2024-01-07  43943.097656  0.078225  6.895833  19.235271  2222.865967  ...   
2024-01-08  46970.503906  0.081370  7.477948  20.332895  2333.032715  ...   
2024-01-09  46139.730469  0.079263  7.117210  21.187775  2344.827148  ...   
2024-01-10  46627.777344  0.082942  7.971061  26.218678  2582.103516  ...   

Attribute      Volume                                                  \
Symbols       LEO-USD   LINK-USD    LTC-USD    MATIC-USD       OP-USD   
Date                                                                    
2024-01-06  1257288.0  379600087  285283013  367787806.0  275752845.0   
2024-01-07   881626.0  310835368  259336849  314333289.0  261136888.0   
2024-01-08  1366341.0  553717423  504603252  621855718.0  382661690.0   
2024-01-09   923438.0  546517541  474216123  561069781.0  445327088.0   
2024-01-10  1088572.0  661542902  530257325  781940772.0  794509110.0   

Attribute                                                              
Symbols        SHIB-USD       SOL-USD    TRX-USD WEOS-USD     XRP-USD  
Date                                                                   
2024-01-06  115014702.0  2.235902e+09  207408616      0.0   820534691  
2024-01-07  130490363.0  2.288062e+09  168263086      0.0   901525412  
2024-01-08  213033437.0  4.152448e+09  279910402     54.0  2011066819  
2024-01-09  300226062.0  4.070132e+09  267455370     54.0  1740178857  
2024-01-10  232788975.0  4.216430e+09  314121921     72.0  2223452643  

[5 rows x 132 columns]

In [4]:
#Extract only the closing price 
data_close = data["Close"]
data_close.head()

Symbols,ADA-USD,ATOM-USD,AVAX-USD,BCH-USD,BNB-USD,BTC-USD,DOGE-USD,DOT-USD,ETC-USD,ETH-USD,...,LEO-USD,LINK-USD,LTC-USD,MATIC-USD,OP-USD,SHIB-USD,SOL-USD,TRX-USD,WEOS-USD,XRP-USD
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-01,0.728657,NaN,NaN,2432.540039,8.41461,13657.200195,0.008909,NaN,34.167900,772.640991,...,NaN,0.733563,229.033005,NaN,NaN,NaN,NaN,0.051695,NaN,2.39103
2018-01-02,0.782587,NaN,NaN,2711.000000,8.83777,14982.099609,0.009145,NaN,34.917099,884.443970,...,NaN,0.673712,255.684006,NaN,NaN,NaN,NaN,0.078682,NaN,2.48090
2018-01-03,1.079660,NaN,NaN,2608.689941,9.53588,15201.000000,0.009320,NaN,34.863400,962.719971,...,NaN,0.681167,245.367996,NaN,NaN,NaN,NaN,0.094703,NaN,3.10537
2018-01-04,1.114120,NaN,NaN,2430.179932,9.21399,15599.200195,0.009644,NaN,36.318001,980.921997,...,NaN,0.984368,241.369995,NaN,NaN,NaN,NaN,0.207974,NaN,3.19663
2018-01-05,0.999559,NaN,NaN,2584.479980,14.91720,17429.500000,0.012167,NaN,36.199501,997.719971,...,NaN,0.907486,249.270996,NaN,NaN,NaN,NaN,0.220555,NaN,3.04871


In [5]:
# Create function to calculate returns
def returns(r):
    rets = r.pct_change().dropna()
    df = pd.DataFrame(rets)
    return df

# Apply the function to the data_close dataframe
rets = returns(data_close)
rets.head()

Symbols,ADA-USD,ATOM-USD,AVAX-USD,BCH-USD,BNB-USD,BTC-USD,DOGE-USD,DOT-USD,ETC-USD,ETH-USD,...,LEO-USD,LINK-USD,LTC-USD,MATIC-USD,OP-USD,SHIB-USD,SOL-USD,TRX-USD,WEOS-USD,XRP-USD
Date,,,,,,,,,,,,,,,,,,,,,
2023-05-12,0.028952,0.010696,0.014664,0.012870,0.002949,-0.007252,0.005188,0.011458,0.005866,0.006418,...,-0.005077,0.040053,-0.006635,0.026806,0.073312,0.0,0.033453,0.020545,-0.012201,0.020952
2023-05-13,-0.014663,-0.024052,-0.016507,-0.016143,0.008048,-0.000780,-0.009412,0.001107,-0.005600,-0.006585,...,0.014738,-0.018359,-0.002452,-0.018123,-0.022484,0.0,-0.002058,0.004110,0.005972,-0.013501
2023-05-14,0.015697,0.001830,0.005902,0.019694,0.003372,0.005472,0.002912,0.000468,-0.000024,0.002442,...,0.009453,0.006679,0.039083,0.012204,-0.000992,0.0,0.004056,0.000303,0.003672,0.002252
2023-05-15,-0.008844,-0.010128,0.004185,0.008036,0.004464,0.009731,0.000833,-0.006373,0.009425,0.009469,...,-0.022728,0.014402,0.043409,0.000557,0.004894,0.0,0.002567,0.012118,0.002279,0.003997
2023-05-16,0.000947,0.007137,-0.012594,0.005100,-0.006855,-0.005738,0.009480,-0.003946,-0.001180,0.003616,...,0.000782,0.003162,0.032058,-0.016554,0.006416,0.0,-0.015603,0.004513,0.010670,0.032416


To build the precision matrix, we will need to get the covariance matrix. Please note the difference between covariance matrix and correlation matrix. The former is not normalized and as such, reflects both the strength and direction of linear relationships between pairwise variables in its own individual scale. 

We will find the covariance of the returns data instead of the nominal price.

In [13]:
# Generate covmat from returns
true_cov = rets.cov()
true_cov.head()

Symbols,ADA-USD,ATOM-USD,AVAX-USD,BCH-USD,BNB-USD,BTC-USD,DOGE-USD,DOT-USD,ETC-USD,ETH-USD,...,LEO-USD,LINK-USD,LTC-USD,MATIC-USD,OP-USD,SHIB-USD,SOL-USD,TRX-USD,WEOS-USD,XRP-USD
Symbols,,,,,,,,,,,,,,,,,,,,,
ADA-USD,0.001396,0.000817,0.001035,0.000583,0.000445,0.000476,0.000684,0.000952,0.000759,0.000539,...,0.000030,0.000919,0.000691,0.001006,0.000751,0.000595,0.001116,0.000363,0.000344,0.001259
ATOM-USD,0.000817,0.001093,0.000812,0.000556,0.000371,0.000384,0.000596,0.000806,0.000674,0.000458,...,0.000004,0.000793,0.000588,0.000881,0.000755,0.000643,0.000857,0.000300,0.000259,0.000717
AVAX-USD,0.001035,0.000812,0.001872,0.000569,0.000388,0.000481,0.000816,0.001019,0.000663,0.000448,...,0.000079,0.000950,0.000688,0.000900,0.000869,0.000824,0.001384,0.000354,0.000226,0.000697
BCH-USD,0.000583,0.000556,0.000569,0.002267,0.000362,0.000522,0.000569,0.000575,0.001024,0.000514,...,0.000066,0.000842,0.000954,0.000626,0.000719,0.000598,0.000678,0.000251,0.000167,0.000333
BNB-USD,0.000445,0.000371,0.000388,0.000362,0.000522,0.000249,0.000342,0.000380,0.000384,0.000292,...,0.000046,0.000456,0.000405,0.000531,0.000515,0.000251,0.000456,0.000168,0.000267,0.000414


In [7]:
# Generate the precision matrix from the covmat
true_prec = np.linalg.inv(true_cov)

In [8]:
# Fit the GraphicalLassoCV model with 1000 iterations
est = GraphicalLassoCV(max_iter=1000).fit(rets)
est

/Users/osim/miniforge3/lib/python3.10/site-packages/numpy/core/_methods.py:173: RuntimeWarning: invalid value encountered in subtract
  x = asanyarray(arr - arrmean)


GraphicalLassoCV(max_iter=1000)

In [9]:
# Access the estimated covmat from GraphicalLassoCV
np.around(est.covariance_, decimals=5)

array([[ 1.3900e-03,  5.5000e-04,  7.7000e-04,  4.4000e-04,  1.8000e-04,
         2.1000e-04,  4.2000e-04,  6.8000e-04,  4.9000e-04,  2.7000e-04,
         5.1000e-04,  7.0000e-04, -0.0000e+00,  6.5000e-04,  4.2000e-04,
         7.4000e-04,  5.9000e-04,  4.1000e-04,  8.5000e-04,  1.0000e-04,
         8.0000e-05,  9.9000e-04],
       [ 5.5000e-04,  1.0900e-03,  5.4000e-04,  3.5000e-04,  1.3000e-04,
         1.4000e-04,  3.3000e-04,  5.4000e-04,  4.1000e-04,  2.1000e-04,
         5.1000e-04,  4.8000e-04, -0.0000e+00,  5.2000e-04,  3.3000e-04,
         6.1000e-04,  4.9000e-04,  3.8000e-04,  5.9000e-04,  6.0000e-05,
         6.0000e-05,  4.9000e-04],
       [ 7.7000e-04,  5.4000e-04,  1.8600e-03,  4.4000e-04,  1.5000e-04,
         2.1000e-04,  5.5000e-04,  7.5000e-04,  4.7000e-04,  2.5000e-04,
         6.4000e-04,  5.7000e-04, -0.0000e+00,  6.8000e-04,  4.3000e-04,
         6.4000e-04,  6.0000e-04,  5.6000e-04,  1.1100e-03,  1.1000e-04,
         7.0000e-05,  6.7000e-04],
       [ 4.4000e-04

#### Constructing Graphical Analysis Network

Next, we need to provide the **labeling** for the network. 

In [ ]:
#Import the Graphical Analysis